# 🧠 Claude Memory on Amazon Bedrock
### Build Your First AI with Perfect Memory in 60 Minutes!

<div style="background: #f0f9ff; padding: 20px; border-radius: 8px; border-left: 4px solid #0369a1;">
<strong>What You'll Build:</strong><br>
An AI assistant that remembers everything about you - permanently across different conversations!
</div>

## 📚 Learning Outcomes

By the end of this tutorial, you will:
- ✅ Understand how Claude Sonnet 4.5's memory system works
- ✅ Implement a minimal client for memory operations
- ✅ Store information that persists across conversations
- ✅ Build personalized AI interactions based on memories
- ✅ Feel confident deploying memory-enabled applications

## ⏱️ Time Required
**45 minutes total** (setup and hands-on practice)

## 📋 Tutorial Structure

### Part 1: Connection Test (5 min)
Verify your connection to Claude Sonnet 4.5

### Part 2: Storing Your First Memory (10 min)
Learn how Claude stores information autonomously

### Part 3: Retrieving Memories in New Sessions (10 min)
See memory persistence across independent conversations

### Part 4: Updating and Managing Memories (10 min)
Dynamically update stored information

### Part 5: Building a Personalized Assistant (10 min)
Create context-aware, personalized interactions

---

## 🎯 Before You Begin

### Prerequisites

This tutorial assumes you have:
- ✅ AWS account with Bedrock access enabled
- ✅ Claude Sonnet 4.5 model access approved
- ✅ AWS credentials configured (`aws configure` completed)
- ✅ Python 3.8+ installed
- ✅ boto3 package installed (`pip install boto3`)

**Need setup help?** See the [Quick Start Guide](docs/tutorials/claude-memory-quickstart.md) for detailed AWS setup instructions.

---
# Part 1: Setup 🚀

In this section, we'll set up everything needed to work with Claude's memory system.

## Setup: Minimal Client Code

The following cells contain everything you need to work with Claude's memory system. We'll build it step-by-step:

1. **Imports and AWS Connection** - Connect to Claude via AWS Bedrock
2. **Memory Tool Handler** - Handle Claude's requests to store/read/update memories  
3. **Chat Function** - Manage conversations with automatic tool use loops

In [3]:
import boto3
import json
from datetime import datetime
from pathlib import Path
from botocore.config import Config

# AWS Bedrock configuration
config = Config(
   retries = {
      'total_max_attempts': 1000,
      'mode': 'standard'
   }
)

# Model configuration
MODEL_ID = "global.anthropic.claude-sonnet-4-5-20250929-v1:0"

# AWS session setup
session = boto3.Session(region_name='us-east-1', profile_name='angmas+bedrock-Bedrock')
bedrock_rt = session.client(
    service_name='bedrock-runtime',
    config=config
)

bedrock_ac_control = session.client('bedrock-agentcore-control')
bedrock_ac = session.client('bedrock-agentcore')

## ✅ Prerequisite Validation

Before proceeding with the tutorial, let's verify that your environment is properly configured.

This cell will check:
- Python version (3.8+ required)
- Required packages (boto3, botocore)
- AWS credentials configuration
- Claude Sonnet 4.5 model access
- AWS Bedrock AgentCore access

**Run the cell below to validate your setup.**

In [5]:
# Prerequisite Validation Cell
# Run this to verify your environment is properly configured

import sys
from importlib import import_module

print("🔍 Validating Prerequisites...")
print("=" * 60)

# Track validation results
all_checks_passed = True

# 1. Check Python Version
print("\n1. Python Version Check")
python_version = sys.version_info
if python_version >= (3, 8):
    print(f"   ✅ Python {python_version.major}.{python_version.minor}.{python_version.micro}")
else:
    print(f"   ❌ Python version too old: {python_version.major}.{python_version.minor}")
    print("      Required: Python 3.8 or higher")
    all_checks_passed = False

# 2. Check Required Packages
print("\n2. Required Python Packages")
required_packages = ['boto3', 'botocore']
for package in required_packages:
    try:
        mod = import_module(package)
        version = getattr(mod, '__version__', 'unknown')
        print(f"   ✅ {package} ({version})")
    except ImportError:
        print(f"   ❌ {package} not installed")
        print(f"      Install with: pip install {package}")
        all_checks_passed = False

# 3. Check AWS Credentials
print("\n3. AWS Credentials")
try:
    # Try to get credentials
    credentials = session.get_credentials()
    if credentials:
        print(f"   ✅ AWS credentials found")
        print(f"      Profile: {session.profile_name}")
        print(f"      Region: {session.region_name}")
    else:
        print("   ❌ AWS credentials not found")
        print("      Run: aws configure")
        all_checks_passed = False
except Exception as e:
    print(f"   ❌ Error checking credentials: {str(e)}")
    all_checks_passed = False

# 4. Check Claude Sonnet 4.5 Model Access
print("\n4. Claude Sonnet 4.5 Model Access")
try:
    # Try to invoke the model with a minimal request
    test_body = {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 10,
        "messages": [{"role": "user", "content": [{"type": "text", "text": "Hi"}]}]
    }
    response = bedrock_rt.invoke_model(
        modelId=MODEL_ID,
        body=json.dumps(test_body)
    )
    print(f"   ✅ Model access confirmed: {MODEL_ID}")
except bedrock_rt.exceptions.AccessDeniedException:
    print(f"   ❌ Access denied to model: {MODEL_ID}")
    print("      Request model access in AWS Bedrock Console:")
    print("      https://console.aws.amazon.com/bedrock/home#/modelaccess")
    all_checks_passed = False
except Exception as e:
    print(f"   ❌ Error testing model: {str(e)}")
    all_checks_passed = False

# 5. Check AgentCore Control Plane Access
print("\n5. AWS Bedrock AgentCore Access")
try:
    # Try to list memories (should work even if empty)
    response = bedrock_ac_control.list_memories(maxResults=1)
    print("   ✅ AgentCore control plane access confirmed")
except Exception as e:
    print(f"   ❌ Cannot access AgentCore: {str(e)}")
    print("      Check: IAM permissions for bedrock-agentcore-control")
    all_checks_passed = False

# Final Summary
print("\n" + "=" * 60)
if all_checks_passed:
    print("🎉 All prerequisites validated successfully!")
    print("✅ You're ready to proceed with the tutorial.")
else:
    print("⚠️  Some prerequisites failed validation.")
    print("❌ Please fix the issues above before continuing.")
    print("\n💡 Common Solutions:")
    print("   • Install packages: pip install boto3")
    print("   • Configure AWS: aws configure")
    print("   • Request model access: AWS Console → Bedrock → Model Access")
    print("   • Check IAM permissions for Bedrock services")
print("=" * 60)

🔍 Validating Prerequisites...

1. Python Version Check
   ✅ Python 3.12.11

2. Required Python Packages
   ✅ boto3 (1.40.41)
   ✅ botocore (1.40.41)

3. AWS Credentials
   ✅ AWS credentials found
      Profile: angmas+bedrock-Bedrock
      Region: us-east-1

4. Claude Sonnet 4.5 Model Access
   ✅ Model access confirmed: global.anthropic.claude-sonnet-4-5-20250929-v1:0

5. AWS Bedrock AgentCore Access
   ✅ AgentCore control plane access confirmed

🎉 All prerequisites validated successfully!
✅ You're ready to proceed with the tutorial.


## Building the Memory Tool Handler

Now that we have our basic connection set up, we need to create a function that handles Claude's memory tool requests.

### What is the Memory Tool?

Claude Sonnet 4.5 has built-in memory capabilities - it can autonomously decide to **store**, **read**, and **update** information about you. When Claude wants to use memory, it sends tool requests to your application with commands like:
- `view` - Read existing memories
- `create` - Store new memories
- `str_replace` - Update existing memories

### Why Do We Need a Handler?

Claude doesn't directly manage the memory - instead, it makes tool requests that we need to implement. Think of it like this:
- **Claude (the brain)**: "I want to remember that this user loves Python"
- **Our handler (the hands)**: Writes that information into AgentCore Memory

### How It Works

Our `handle_memory_tool()` function translates Claude's requests into AgentCore Memory operations:
1. **List memories**: When Claude asks to `view /memories`, we return all the events stored in the memory.
2. **Read a memory**: When Claude asks for a specific memory file (e.g., `/memories/user_profile.md`), we find the relevant memory and return its contents
3. **Create memory**: When Claude wants to remember something new, we create a new event in AgentCore Memory and store the information specified by Claude
4. **Update memory**: When Claude wants to update existing information, we create a new event with the same name with the modified content and we remove the old event.

**Note**: Claude will use `.md` name extensions directly thanks to a system prompt we'll configure later in the chat function. This means our handler can use paths exactly as Claude provides them - no conversion needed! While this is not necessary for the AgentCore Memory implementation we keep this convention as it might help Claude define about the memory format.

Let's first create the memory:

## Create the memory

For this example we are going to use the Short Term Memory feature of AgentCore Memory. AgentCore Memory provides both Short Term and Long Term memories. Short Term memory allows us to directly create and delete events, while Long Term memories are extracted automatically by AgentCore from the short term memory events.

In [6]:
memory = bedrock_ac_control.create_memory(name="claude_memory", eventExpiryDuration=7)

Memories are scoped to actors and sessions. In this sample we set a fixed session id and actor id. In a real application, the actor id will represent the user id interacting with the application and the session id can be used to create session specific memories, that are persisted and recoverable at any time before the event expiry duration. This allow to implement a session resume functionality for example. 

In [7]:
MEMORY=memory['memory']
SESSION_ID="0000-0000"
ACTOR = 'user_123'

Let's now implement this handler. AgentCore Memory supports 2 types of events: conversational and blob. Since the content of the memory is determined by Claude we are going to use the    `blob` type to mimic a key value storage. What is nice with AgentCore memory is that you pay only for the events when they are created and reading is free. 

Note also that `blob` can only store strings, so we serialize our JSON object as string.

For the structure to store we chose to use:

```json
{
    "name": "the name of the memory",
    "text": "the markdown content"
}
```

In [8]:
from datetime import datetime
from uuid import uuid4

print("AWS Bedrock connection ready")
print(f"AgentCore Memory: {MEMORY['id']}")

def handle_memory_tool(tool_input):
    """File-based handler for Claude Sonnet 4.5's memory tool requests with improved error handling"""
    try:
        command = tool_input.get('command')
        path = tool_input.get('path', '')
        
        if not command:
            return {'error': 'Missing required field: command'}
        
        if command == 'view':
            if path == '/memories':
                # List all memory files
                try:
                    memories = []
                    events = bedrock_ac.list_events(
                        memoryId=MEMORY['id'], 
                        actorId=ACTOR, 
                        sessionId=SESSION_ID
                    )['events']
                    
                    for event in events:
                        try:
                            file = json.loads(event['payload'][0]['blob'])
                            memories.append({
                                'path': f"/memories/{file['name']}",
                                'content': file['text'],
                                'created': event['eventTimestamp'].isoformat()
                            })
                        except (json.JSONDecodeError, KeyError) as e:
                            print(f"   ⚠️ Warning: Skipping malformed event: {e}")
                            continue
                    
                    return {'memories': memories}
                except Exception as e:
                    return {'error': f'Failed to list memories: {str(e)}'}
                    
            elif path.startswith('/memories/'):
                # Get specific memory file
                filename = path.split('/')[-1]
                try:
                    events = bedrock_ac.list_events(
                        memoryId=MEMORY['id'], 
                        actorId=ACTOR, 
                        sessionId=SESSION_ID
                    )['events']
                    
                    for event in events:
                        try:
                            file = json.loads(event['payload'][0]['blob'])
                            if file['name'] == filename:
                                return {
                                    'memory': {
                                        'path': f"/memories/{file['name']}",
                                        'content': file['text'],
                                        'created': event['eventTimestamp'].isoformat()
                                    }
                                }
                        except (json.JSONDecodeError, KeyError) as e:
                            print(f"   ⚠️ Warning: Skipping malformed event: {e}")
                            continue
                    
                    return {'memory': {'error': 'not found'}}
                except Exception as e:
                    return {'error': f'Failed to read memory: {str(e)}'}
        
        elif command == 'create':
            # Create new memory file
            file_text = tool_input.get('file_text', '')
            
            # Use UUID-based naming for better performance (no need to count all events)
            if '/' in path and path.split('/')[-1]:
                filename = path.split('/')[-1]
            else:
                filename = f"mem_{uuid4().hex[:8]}.md"
            
            try:
                bedrock_ac.create_event(
                    memoryId=MEMORY['id'], 
                    actorId=ACTOR, 
                    sessionId=SESSION_ID,
                    payload=[{
                        'blob': json.dumps({
                            'name': filename,
                            'text': file_text,
                        })
                    }],
                    eventTimestamp=datetime.now()
                )
                print(f"   💾 Stored memory file: {filename}")
                return {'success': True, 'created': filename}
            except Exception as e:
                return {'error': f'Failed to create memory: {str(e)}'}
        
        elif command == 'str_replace':
            # Update existing memory file
            filename = path.split('/')[-1]
            old_str = tool_input.get('old_str', '')
            new_str = tool_input.get('new_str', '')
            
            if not old_str:
                return {'error': 'Missing required field: old_str'}
            
            try:
                events = bedrock_ac.list_events(
                    memoryId=MEMORY['id'], 
                    actorId=ACTOR, 
                    sessionId=SESSION_ID
                )['events']
                
                for event in events:
                    try:
                        file = json.loads(event['payload'][0]['blob'])
                        if file['name'] == filename:
                            # Validate that old_str exists in content
                            if old_str not in file['text']:
                                return {'error': f'String to replace not found: {old_str[:50]}...'}
                            
                            # Perform replacement
                            updated_content = file['text'].replace(old_str, new_str)
                            
                            # Create new event with updated content
                            try:
                                bedrock_ac.create_event(
                                    memoryId=MEMORY['id'], 
                                    actorId=ACTOR, 
                                    sessionId=SESSION_ID,
                                    payload=[{
                                        'blob': json.dumps({
                                            'name': filename,
                                            'text': updated_content,
                                        })
                                    }],
                                    eventTimestamp=datetime.now()
                                )
                                
                                # Only delete old event after successful creation
                                event_copy = dict(event)
                                for key in ['eventTimestamp', 'payload', 'branch']:
                                    event_copy.pop(key, None)
                                bedrock_ac.delete_event(**event_copy)
                                
                                print(f"   ✏️ Updated memory file: {filename}")
                                return {'success': True}
                            except Exception as e:
                                return {'error': f'Failed to update memory: {str(e)}'}
                    except (json.JSONDecodeError, KeyError) as e:
                        print(f"   ⚠️ Warning: Skipping malformed event: {e}")
                        continue
                
                return {'error': 'Memory not found'}
            except Exception as e:
                return {'error': f'Failed to update memory: {str(e)}'}
        
        return {'status': 'handled', 'command': command}
    
    except Exception as e:
        return {'error': f'Unexpected error in memory tool: {str(e)}'}

print("✅ Memory handler function ready (with improved error handling)")

AWS Bedrock connection ready
AgentCore Memory: claude_memory-Lh8jTf3OY2
✅ Memory handler function ready (with improved error handling)


## Creating the Chat Function with Tool Support

Now we have a handler for memory operations, but we need a way to actually **talk to Claude** and handle its tool requests automatically.

### Why Not Just Call the API Directly?

Claude's memory system uses a **tool use loop**. Here's what happens:
1. You send a message to Claude
2. Claude might respond with text OR with a tool use request (to access memory)
3. If it requests a tool, you call `handle_memory_tool()` and send the result back
4. Claude might then request MORE tools, or finally respond with text
5. Repeat until Claude gives a final text response

This back-and-forth needs to happen automatically - you don't want to manually check for tool requests each time.

### What Does chat_with_claude() Do?

Our `chat_with_claude()` function handles this entire loop:
- **Manages conversation history**: Keeps track of all messages (yours and Claude's)
- **Detects tool requests**: Automatically checks if Claude wants to use the memory tool
- **Calls our handler**: Routes tool requests to `handle_memory_tool()`
- **Loops until complete**: Keeps going until Claude gives a final text response

### System Prompt Configuration

To ensure Claude uses `.md` file extensions (instead of defaulting to `.txt`), we'll configure a system prompt:

```python
SYSTEM_PROMPT = "When using the memory tool, always use .md file extensions for markdown-formatted memories."
```

**Why use `.md` extensions?**
- Markdown files render nicely in editors and GitHub
- Supports formatting like **bold**, *italics*, lists, code blocks
- Better readability for human review

This system prompt will be included in every API call to Claude, ensuring consistent `.md` usage throughout.

### The Tool Use Loop in Action

```text
You: "Remember that I love Python"
  → Claude: [tool_use: create memory about Python preference]
    → Handler: Creates { "name":"user_preferences.md", "text":...} event
    → [tool_result: success]
  → Claude: "I've stored that you love Python. I'll remember this!"
```

Note that this function is generic and does not depend on the actual memory tool implementation. The same function is used in the [claude_memory_tutorial.ipynb](./claude_memory_tutorial.ipynb) notebook.

Let's implement this chat function with automatic tool handling:

In [9]:
SYSTEM_PROMPT = "When using the memory tool, always use .md file extensions for markdown-formatted memories."

def chat_with_claude(message, conversation_history=None):
    """Send a message to Claude Sonnet 4.5 and handle any tool use"""
    messages = conversation_history or []
    
    # Add user message
    messages.append({
        "role": "user",
        "content": [{"type": "text", "text": message}]
    })
    
    print(f"\n👤 You: {message}\n")
    
    # Keep handling tool uses until Claude Sonnet 4.5 stops requesting them
    max_iterations = 5  # Safety limit
    iteration = 0
    
    while iteration < max_iterations:
        iteration += 1
        
        # Prepare request body
        body = {
            "anthropic_version": "bedrock-2023-05-31",
            "anthropic_beta": ["context-management-2025-06-27"],
            "system": [{"type": "text", "text": SYSTEM_PROMPT}],
            "tools": [{
                "type": "memory_20250818",
                "name": "memory"
            }],
            "max_tokens": 4000,
            "messages": messages,
        }
        
        # Send to Claude Sonnet 4.5
        response = bedrock_rt.invoke_model(
            modelId=MODEL_ID,
            body=json.dumps(body)
        )
        result = json.loads(response["body"].read())
        
        # Track if we need to handle tool use
        has_tool_use = False
        tool_uses = []
        
        # Process response
        for content in result.get('content', []):
            if content['type'] == 'text':
                print(f"🤖 Claude: {content['text']}\n")
            
            elif content['type'] == 'tool_use':
                has_tool_use = True
                tool_uses.append(content)
                print(f"🔧 Claude uses memory tool: {content['input'].get('command', 'unknown')}")
                print(f"   Path: {content['input'].get('path', 'N/A')}")
                if content['input'].get('file_text'):
                    print(f"   Creating memory file...\n")
        
        # Add Claude's response to conversation
        messages.append({
            "role": "assistant",
            "content": result['content']
        })
        
        # If no tool use, we're done
        if not has_tool_use:
            break
        
        # Handle all tool uses
        tool_results = []
        for tool_use in tool_uses:
            tool_result = handle_memory_tool(tool_use['input'])
            tool_results.append({
                "type": "tool_result",
                "tool_use_id": tool_use['id'],
                "content": json.dumps(tool_result)
            })
        
        # Add tool results to conversation
        messages.append({
            "role": "user",
            "content": tool_results
        })
        
        # Continue loop to get Claude's next response
    
    return messages

print("✅ Chat function ready")

✅ Chat function ready


### 🎯 Checkpoint: Setup Complete!

**What you just did:**
- ✅ Imported required libraries
- ✅ Connected to AWS Bedrock
- ✅ Created memory directory for file-based storage
- ✅ Built memory tool handler to translate Claude's requests into file operations
- ✅ Built chat function with automatic tool use loop

**Progress**: [█░░░░] 20% complete

### What You've Built: The Complete Architecture

```text
┌──────────────────────────────────────────────────────────────┐
│                     Your Application                         │
│                                                              │
│  ┌──────────────────┐    ┌──────────────────┐                │
│  │  chat_with_      │───▶│  handle_memory_  │                │
│  │  claude()        │◀───│  tool()          │                │
│  │                  │    │                  │                │
│  │ - Sends messages │    │ - view memories  │                │
│  │ - Handles tool   │    │ - create files   │    ┌─────────┐ │
│  │   use loop       │    │ - update files   │───▶│memories/│ │
│  │ - System prompt  │    │                  │    │ *.md    │ │
│  └──────────────────┘    └──────────────────┘    └─────────┘ │
│           │                                                  │
│           ▼                                                  │
│  ┌──────────────────┐                                        │
│  │  AWS Bedrock     │                                        │
│  │  Claude Sonnet   │                                        │
│  │  4.5             │                                        │
│  └──────────────────┘                                        │
└──────────────────────────────────────────────────────────────┘
```

**Key Insight**: Claude autonomously decides when to use memory. You just provide the tools!

You're ready to store your first memory!

---
# Part 2: Storing Your First Memory 📝

## What You'll Learn
How Claude's memory system works and how to store information autonomously.

## Your Goal
Store information about yourself that Claude can recall later.

## 💡 Key Concept: The Memory System

Think of it like this:
- 📝 **Claude** decides what to write in a notebook
- 📂 **Your code** provides the notebook and retrieves pages when asked
- 🔄 **Memories** persist as event in Agent Core Memory

You don't tell Claude what to remember - it figures that out automatically!

In [10]:
print("=" * 60)
print("SESSION 1: Teaching Claude About You")
print("=" * 60)

# Tell Claude about yourself
conversation1 = chat_with_claude(
    "Hi Claude! Please remember these things about me: "
    "I'm a Python developer who loves machine learning, "
    "I use VSCode with dark theme, "
    "and my favorite libraries are numpy, pandas, and scikit-learn."
)

SESSION 1: Teaching Claude About You

👤 You: Hi Claude! Please remember these things about me: I'm a Python developer who loves machine learning, I use VSCode with dark theme, and my favorite libraries are numpy, pandas, and scikit-learn.

🔧 Claude uses memory tool: view
   Path: /memories
🔧 Claude uses memory tool: create
   Path: /memories/user_profile.md
   Creating memory file...

   💾 Stored memory file: user_profile.md
🤖 Claude: 

Perfect! I've saved your profile information to my memory. I now know that you're a Python developer who loves machine learning, you use VSCode with a dark theme, and your favorite libraries are numpy, pandas, and scikit-learn. This information will help me provide more relevant and personalized assistance in our future conversations! 🐍✨



### 💡 Pro Tip: Viewing Memory Files

Since these are events stored in Agent Core Memory you can:
- Use `list_events` to list them
- Read them with `get_event`

Let's view what was created!

In [11]:
print("📦 Memory Files Created:")
print("=" * 60)

events = bedrock_ac.list_events(memoryId=MEMORY['id'], actorId=ACTOR, sessionId='0000-0000')['events']
memory_files = [{**json.loads(event['payload'][0]['blob']), "created": event['eventTimestamp']} for event in events]
print(f"Total memory files: {len(memory_files)}")
print(f"Memory ID: {MEMORY['id']}\n")

for md_file in memory_files:
    print(f"File: {md_file['name']}")
    print(f"Size: {len(md_file['text'])} bytes")
    print(f"Created: {md_file['created'].isoformat()}")
    
    content = md_file['text']
    if content:
        preview = content[:200] + "..." if len(content) > 200 else content
        print(f"\nContent preview:\n{preview}")
    print("-" * 60)

if len(memory_files) > 0:
    print("\n🎉 SUCCESS! Claude stored its first memory as a .md file!")
    print(f"💡 You can open these files in any text editor!")
else:
    print("\n⚠️ No memory files created. Try running the previous cell again.")

📦 Memory Files Created:
Total memory files: 1
Memory ID: claude_memory-Lh8jTf3OY2

File: user_profile.md
Size: 231 bytes
Created: 2025-10-06T10:22:26+01:00

Content preview:
# User Profile

## Professional Background
- **Role**: Python developer
- **Interests**: Machine learning

## Development Environment
- **IDE**: VSCode
- **Theme**: Dark theme

## Favorite Libraries
-...
------------------------------------------------------------

🎉 SUCCESS! Claude stored its first memory as a .md file!
💡 You can open these files in any text editor!


## Session 2: Test Memory Recall

Let's start a completely fresh conversation with no history.

In [12]:
print("=" * 60)
print("SESSION 2: New Conversation - Testing Memory Recall")
print("=" * 60)
print("🔄 Starting fresh - NO conversation history passed\n")

# Completely new conversation - Claude should remember from memory
conversation2 = chat_with_claude(
    "What do you remember about me and my interests?"
)

SESSION 2: New Conversation - Testing Memory Recall
🔄 Starting fresh - NO conversation history passed


👤 You: What do you remember about me and my interests?

🔧 Claude uses memory tool: view
   Path: /memories
🤖 Claude: 

Based on my memory, here's what I know about you:

**Professional Background:**
- You're a **Python developer** with an interest in **machine learning**

**Development Setup:**
- You use **VSCode** as your IDE
- You prefer a **dark theme**

**Favorite Libraries:**
- numpy
- pandas
- scikit-learn

Is there anything you'd like to add or update about yourself? I'm always happy to learn more about your interests, projects, or preferences!



### 🔍 Understanding Check

**What just happened?**
1. You started a new conversation with no context
2. Claude used the memory tool to VIEW stored memories
3. It retrieved the information from AgentCore Memory
4. It responded with personalized information about you

**This is different from regular chat history!**
- Regular chat: Context passed in every message (limited to session)
- Memory: Stored separately in AgentCore memory, flexible session scoping

## Session 3: Personalized Recommendation

Let's ask Claude to use its memory to give us personalized recommendations.

In [13]:
print("=" * 60)
print("SESSION 3: Personalized Recommendation")
print("=" * 60)
print("🎯 Testing if Claude uses memories for personalization\n")

conversation3 = chat_with_claude(
    "Based on what you know about me, recommend a Python library "
    "I might like for data visualization."
)

SESSION 3: Personalized Recommendation
🎯 Testing if Claude uses memories for personalization


👤 You: Based on what you know about me, recommend a Python library I might like for data visualization.

🤖 Claude: I'll check my memory to see what I know about you first.

🔧 Claude uses memory tool: view
   Path: /memories
🤖 Claude: 

Great! Based on your profile, I can see you're a Python developer interested in machine learning, and you already use numpy, pandas, and scikit-learn.

## Recommendation: **Plotly**

Given your background, I'd recommend **Plotly** for data visualization. Here's why it's a great fit for you:

### Why Plotly?
1. **Interactive visualizations** - Perfect for exploring ML model results, feature distributions, and performance metrics
2. **Integrates seamlessly** with pandas DataFrames and numpy arrays (your existing stack)
3. **ML-specific features**:
   - Confusion matrices
   - ROC curves and AUC visualization
   - Feature importance plots
   - Dimensionality reduc

### 🎯 Checkpoint: Part 3 Complete!

You've successfully:
- ✅ Started new conversations without context
- ✅ Verified memory persistence across sessions
- ✅ Seen personalized responses based on memories

**Progress**: [███░░] 60% complete

🎊 **AMAZING!** Claude remembered across independent sessions!

---
# Part 4: Updating and Managing Memories ✏️

## What You'll Learn
How Claude dynamically updates memories as new information becomes available.

## Your Goal
Update existing memories with new preferences and verify the changes.

## Session 4: Update Your Preferences

Let's tell Claude about some changes in your preferences.

In [14]:
print("=" * 60)
print("SESSION 4: Updating Memories")
print("=" * 60)
print("✏️ Telling Claude about preference changes\n")

conversation4 = chat_with_claude(
    "Update your memory about me: I've recently started using PyTorch "
    "for deep learning and I'm also interested in transformer models. "
    "Also, I now prefer using Jupyter Lab instead of just VSCode."
)

SESSION 4: Updating Memories
✏️ Telling Claude about preference changes


👤 You: Update your memory about me: I've recently started using PyTorch for deep learning and I'm also interested in transformer models. Also, I now prefer using Jupyter Lab instead of just VSCode.

🤖 Claude: I'll check my memory first and then update it with your new information.

🔧 Claude uses memory tool: view
   Path: /memories
🤖 Claude: 

Now I'll update your profile with the new information about PyTorch, transformer models, and your preference for Jupyter Lab.

🔧 Claude uses memory tool: str_replace
   Path: /memories/user_profile.md
   ✏️ Updated memory file: user_profile.md
🔧 Claude uses memory tool: str_replace
   Path: /memories/user_profile.md
   ✏️ Updated memory file: user_profile.md
🔧 Claude uses memory tool: str_replace
   Path: /memories/user_profile.md
   ✏️ Updated memory file: user_profile.md
🤖 Claude: 

Perfect! I've updated my memory about you with the following changes:

1. ✅ Added **deep l

### 🔍 What to Look For

Claude may:
- Use `str_replace` to update existing memory event. 
- OR create a new memory event
- OR append to existing content


**NOTE**: AgentCore Memory events are immutable: to edit a memory we create e new one with the modified content and remove the old one.

Claude decides the best approach autonomously!

## Session 5: Verify the Update

Let's check if the memory was actually updated.

In [15]:
print("=" * 60)
print("Verifying Memory Update")
print("=" * 60)

conversation5 = chat_with_claude(
    "What do you know about my deep learning interests?"
)

Verifying Memory Update

👤 You: What do you know about my deep learning interests?

🤖 Claude: I'll check my memory to see what I know about your deep learning interests.

🔧 Claude uses memory tool: view
   Path: /memories
🤖 Claude: 

Based on my memory, here's what I know about your deep learning interests:

**Deep Learning Interests:**
- You're interested in **deep learning** in general
- Specifically interested in **transformer models**
- You've recently started using **PyTorch** for deep learning work

**Related Technical Profile:**
- You're a Python developer
- Also interested in machine learning more broadly
- Your favorite libraries include numpy, pandas, and scikit-learn (the ML fundamentals)
- You prefer working in Jupyter Lab or VSCode with a dark theme

Is there anything specific about deep learning you'd like to discuss or explore further? Or would you like me to record any additional interests or projects you're working on?



## View All Memories

Let's look at the complete state of our memory store.

In [16]:
print("📦 Memories Created:")
print("=" * 60)

events = bedrock_ac.list_events(memoryId=MEMORY['id'], actorId=ACTOR, sessionId='0000-0000')['events']
memory_files = [{**json.loads(event['payload'][0]['blob']), "created": event['eventTimestamp']} for event in events]
print(f"Total memory files: {len(memory_files)}")
print(f"Memory ID: {MEMORY['id']}\n")

for md_file in memory_files:
    print(f"File: {md_file['name']}")
    print(f"Size: {len(md_file['text'])} bytes")
    print(f"Created: {md_file['created'].isoformat()}")
    
    content = md_file['text']
    if content:
        preview = content[:500] + "..." if len(content) > 500 else content
        print(f"\nContent preview:\n{preview}")
    print("-" * 60)

if len(memory_files) > 0:
    print("\n🎉 SUCCESS! Claude stored its first memory as a .md file!")
    print(f"💡 You can open these files in any text editor!")
else:
    print("\n⚠️ No memory files created. Try running the previous cell again.")

📦 Memories Created:
Total memory files: 1
Memory ID: claude_memory-Lh8jTf3OY2

File: user_profile.md
Size: 344 bytes
Created: 2025-10-06T10:24:59+01:00

Content preview:
# User Profile

## Professional Background
- **Role**: Python developer
- **Interests**: Machine learning, deep learning, transformer models

## Development Environment
- **IDE**: Jupyter Lab (preferred), VSCode
- **Theme**: Dark theme

## Favorite Libraries
- numpy
- pandas
- scikit-learn
- PyTorch (recently started using for deep learning)

------------------------------------------------------------

🎉 SUCCESS! Claude stored its first memory as a .md file!
💡 You can open these files in any text editor!


In [65]:
print("📊 Memory Usage Analytics")
print("=" * 60)

events = acd.list_events(memoryId=MEMORY['id'], actorId=ACTOR, sessionId='0000-0000')['events']
memory_files = [{**json.loads(event['payload'][0]['blob']), "created": event['eventTimestamp']} for event in events]
total_memories = len(memory_files)
total_size = sum([len(f['text']) for f in memory_files])

print(f"Total memory files: {total_memories}")
print(f"Total storage used: {total_size} bytes ({total_size/1024:.2f} KB)")
print(f"Memory ID: {MEMORY['id']}")
print(f"\nMemory file names: {[f['name'] for f in memory_files]}")

print("\n📅 Memory Timeline:")
for md_file in sorted(memory_files, key=lambda f: f['created']):
    created = md_file['created'].isoformat()[:19]
    size = len(md_file['text'])
    print(f"  {created} - {md_file['name']} ({size} bytes)")

print("\n" + "=" * 60)
print("🏆 CONGRATULATIONS! Tutorial Complete!")
print("=" * 60)
print(f"\n💡 Your memory files are saved in: {MEMORY['id']}")
print("💡 You can open them in any text editor to view or edit them!")
print("💡 They persist across kernel restarts and notebook sessions!")

📊 Memory Usage Analytics
Total memory files: 1
Total storage used: 431 bytes (0.42 KB)
Memory ID: claude_memory-1ng0BT47t6

Memory file names: ['user_profile.md']

📅 Memory Timeline:
  2025-10-03T10:54:37 - user_profile.md (431 bytes)

🏆 CONGRATULATIONS! Tutorial Complete!

💡 Your memory files are saved in: claude_memory-1ng0BT47t6
💡 You can open them in any text editor to view or edit them!
💡 They persist across kernel restarts and notebook sessions!


### 🎯 Checkpoint: Part 5 Complete!

You've successfully:
- ✅ Generated personalized content
- ✅ Tested cross-session memory
- ✅ Analyzed memory usage
- ✅ Built a complete memory-enabled system

**Progress**: [█████] 100% complete

# 🏆 TUTORIAL COMPLETE!

You're now a Claude Memory Expert!

---
# 🎓 What You've Accomplished

## Skills Mastered

✅ **Memory System Architecture**
- Understand client vs. Claude roles
- Know how tool use patterns work
- Grasp memory persistence concepts

✅ **Implementation Skills**
- Set up AWS Bedrock with Claude Sonnet 4.5
- Implement memory storage layer
- Handle tool use requests
- Manage conversation loops

✅ **Practical Applications**
- Store information across sessions
- Retrieve memories contextually
- Update memories dynamically
- Build personalized interactions

## Real-World Use Cases

You can now build:
- 💬 Personalized chatbots that remember user preferences
- 🤝 AI assistants for long-term projects
- 📚 Learning systems that track progress
- 🎯 Context-aware recommendation engines
- 💼 Customer service bots with persistent context

---
# 🚀 Next Steps

## Immediate Next Steps

1. **Explore Your Memory Files**
   - Open the `memories/` directory in your file browser
   - Edit a `.md` file directly and see Claude use the updated content
   - Try version controlling them with git

2. **Test Persistence**
   - Restart the kernel and re-run from cell-1
   - Your memory files will still be there!
   - Claude will load them automatically

3. **Build a Simple App**
   - Create a command-line chatbot
   - Add it to a Discord bot
   - Integrate with a web application

## Advanced Explorations

**Production Memory Systems**
- Database integration (PostgreSQL, DynamoDB)
- Cloud storage (S3, GCS) for `.md` files
- Memory size limits and cleanup policies
- Multi-user support with user-specific directories

**Advanced Patterns**
- Semantic memory search with embeddings
- Automatic memory summarization
- Context-aware memory loading
- Memory conflict resolution

**Integration Projects**
- Flask/FastAPI web applications
- Discord/Slack bots
- Voice assistants
- Mobile app backends

## Learning Resources

📖 **Documentation**
- [Complete Setup Guide](docs/tutorials/claude-memory-quickstart.md)
- [AWS Bedrock Documentation](https://docs.aws.amazon.com/bedrock/)
- [Anthropic API Docs](https://docs.anthropic.com/)

💡 **Try These Challenges**
1. Build a bot that learns your daily schedule
2. Create a study assistant that tracks what you've learned
3. Make a recipe recommender that remembers your preferences
4. Build a project manager that remembers team members' skills

---
# 📚 Additional Resources

## Troubleshooting

**Common Issues:**
- Memory not persisting? Check that `memory_store` isn't being reset
- AWS errors? Verify credentials and model access
- Tool use not working? Confirm `anthropic_beta` flag is set

**Get Help:**
- See detailed troubleshooting: `docs/tutorials/claude-memory-quickstart.md#troubleshooting`
- Check AWS CloudWatch logs for detailed errors
- Review the verification script in Part 1

## Code Reference

Save this notebook as your reference! You can use it as a template for your own projects.

## Share Your Creation

Built something cool with Claude's memory? Share it with the community!

---

# 🎉 Congratulations!

You've completed the Claude Memory tutorial and are ready to build amazing memory-enabled AI applications!

**Remember**: Claude manages the intelligence of memory - you just provide the storage. This simple division of responsibility enables incredibly powerful applications.

Happy building! 🚀

---

# 🧹 Cleanup (Optional)

When you're done with the tutorial, you can clean up the resources to avoid any potential charges.

## What Gets Cleaned Up

- **Memory Store**: Deletes the AgentCore memory and all stored events
- **Memory Events**: All stored memory files will be removed from AWS

⚠️ **Warning**: This action is permanent and cannot be undone. Only run this if you're completely finished with the tutorial.

**Run the cell below to clean up resources.**

In [18]:
# Cleanup Script - Run this to delete the memory and all events

import time

print("🧹 Starting Cleanup Process...")
print("=" * 60)

try:
    # Get the memory ID
    memory_id = MEMORY['id']
    print(f"\n📦 Memory to clean up: {memory_id}")
    
    print("\n2️⃣ Deleting memory store...")
    try:
        bedrock_ac_control.delete_memory(memoryId=memory_id)
        print(f"   ✅ Memory {memory_id} deleted successfully")
    except Exception as e:
        print(f"   ❌ Error deleting memory: {str(e)}")
    
    print("\n" + "=" * 60)
    print("🎉 Cleanup Complete!")
    print("\n💡 Note: You can safely close this notebook now.")
    print("💡 If you want to run the tutorial again, restart the kernel")
    print("   and re-run from the beginning.")
    print("=" * 60)
    
except NameError:
    print("\n❌ Error: MEMORY variable not found.")
    print("   This usually means the memory was never created.")
    print("   Nothing to clean up!")
except Exception as e:
    print(f"\n❌ Unexpected error during cleanup: {str(e)}")
    print("   You may need to manually delete resources in AWS Console.")

🧹 Starting Cleanup Process...

📦 Memory to clean up: claude_memory-Lh8jTf3OY2

2️⃣ Deleting memory store...
   ✅ Memory claude_memory-Lh8jTf3OY2 deleted successfully

🎉 Cleanup Complete!

💡 Note: You can safely close this notebook now.
💡 If you want to run the tutorial again, restart the kernel
   and re-run from the beginning.
